In [29]:
import os 
import glob
from ase.io import read,write
from ovito.io import import_file
import WarrenCowleyParameters as wc
import seaborn as sns
import pandas as pd

def progress_print(i, total, comment=''):
    """
    Print progress percentage and current file number being processed.
    
    Parameters
    ----------
    i : int
        Current iteration index (0-based).
    total : int
        Total number of items to process.
    
    Returns
    -------
    None
        Prints progress to stdout with carriage return for dynamic updating.
    """
    progress = (i + 1) / total * 100
    print(f'{comment} Progress: {progress:.0f}% \t Processing File {i+1}/{total}', end='\r', flush=True)


simulation_folder = '/nfshome/winkelmann/CathodeSimulationResults/Lichtenberg_X-mas_25_26'

# find Ovito Files

In [2]:
pattern = os.path.join(simulation_folder, '**', '*.XDATCAR')
path_all_ovito_files = sorted(glob.glob(pattern, recursive=True))

# Find all truncated files and create list of their original files
truncated_files = [f for f in path_all_ovito_files if f.endswith('_truncated.XDATCAR')]
original_files_to_exclude = []#[f.replace('_truncated.XDATCAR', '.XDATCAR') for f in truncated_files]

# Exclude both truncated files and their base files
path_all_ovito_files = [f for f in path_all_ovito_files if f not in truncated_files and f not in original_files_to_exclude]
print(f"Found {len(path_all_ovito_files)} ovito files to process.")
print(f'excluded {len(truncated_files)} truncated files and their base files.')
print('\n'.join(f'{i+1}. {f}' for i, f in enumerate(sorted(original_files_to_exclude))))

Found 90 ovito files to process.
excluded 0 truncated files and their base files.



# truncate to only Li and X Atoms

In [ ]:
#todo: find a way to keep the labels of the frames
def truncate_ovito_files(path_ovito_file, wanted_species=['Li','X']):
    with open(path_ovito_file, 'r') as file:
        lines = file.readlines()
    step, time = [], []
    for line in lines:
        if 'step' in line:
            words = line.split(' ')
            step.append(int(words[1]))
            time.append(float(words[3]))
    
    all_frames = read(path_ovito_file, index=':')
    truncated_ovito_file = []
    for n,atoms in enumerate(all_frames):  
        indices = [i for i, atom in enumerate(atoms) if atom.symbol in wanted_species]
        new_atoms = atoms[indices]
        new_atoms.info = atoms.info
        new_atoms.write(filename=path_ovito_file.replace('.XDATCAR', f'_truncated.XDATCAR'),
                        format='vasp-xdatcar', append=True, label=f'step {step[n]} time {time[n]}')
        truncated_ovito_file.append(new_atoms)      
    #write(path_ovito_file.replace('.XDATCAR', '_truncated.XDATCAR'), truncated_ovito_file)
    return truncated_ovito_file

for i, path_ovito_file in enumerate(path_all_ovito_files):
    progress_print(i, len(path_all_ovito_files))
    truncate_ovito_files(path_ovito_file)

# find trunicated files ()

In [4]:
pattern = os.path.join(simulation_folder, '**', '*_truncated.XDATCAR')
path_truncated_files = sorted(glob.glob(pattern, recursive=True))
print(f"Found {len(path_truncated_files)} truncated files to process.")

# calculate Warren Cowley and safe it to a CSV

In [ ]:
def create_warren_cowley_csv(structure_file):
    
    with open(structure_file, 'r') as file:
        lines = file.readlines()
    step, time = [], []
    for line in lines:
        if 'step' in line:
            words = line.split(' ')
            step.append(int(words[1]))
            time.append(float(words[3]))
        
    ovito_file = import_file(structure_file)
    
    num_frames = ovito_file.source.num_frames
    mod = wc.WarrenCowleyParameters(nneigh=[0, 6])
    ovito_file.modifiers.append(mod)
    
    data = ovito_file.compute(0)
    wc_names = list(data.attributes['Warren-Cowley parameters by particle name'][0].keys())
    
    wc_file = structure_file.replace('Ovito_truncated.XDATCAR', '_warren_cowley.csv')
    with open(wc_file, 'w') as f:
        f.write(f"{'step':<7},\t{'time':<15},\t{wc_names[0]:<20},\t{wc_names[1]:<20},\t{wc_names[2]:<20},\t{wc_names[3]:<20}\n")
    print(f'Frames: {num_frames}\t steps:{len(step)}\ttime:{len(time)}')
    # Iterate through frames
    for frame in range(num_frames):
        data = ovito_file.compute(frame)
        wc_for_shells = data.attributes['Warren-Cowley parameters'][0]
        with open(wc_file, 'a') as f:
            f.write(
                f"{step[frame]:7},\t{time[frame]:15},\t{wc_for_shells[0][0]:20},\t{wc_for_shells[0][1]:20},\t"
                f"{wc_for_shells[1][0]:20},\t{wc_for_shells[1][1]:20}\n"
            )
    

for i, path in enumerate(path_truncated_files):
    progress_print(i, len(path_truncated_files))
    create_warren_cowley_csv(path)
    

Frames: 142	 steps:142	time:1421/90
Frames: 142	 steps:142	time:1422/90
Frames: 142	 steps:142	time:1423/90
Frames: 141	 steps:141	time:1414/90
Frames: 143	 steps:143	time:1435/90
Frames: 143	 steps:143	time:1436/90
Frames: 143	 steps:143	time:1437/90
Frames: 142	 steps:142	time:1428/90
Frames: 141	 steps:141	time:1419/90
Frames: 142	 steps:142	time:142 10/90
Frames: 117	 steps:117	time:117 11/90
Frames: 124	 steps:124	time:124 12/90
Frames: 119	 steps:119	time:119 13/90
Frames: 125	 steps:125	time:125 14/90
Frames: 120	 steps:120	time:120 15/90
Frames: 119	 steps:119	time:119 16/90
Frames: 118	 steps:118	time:118 17/90
Frames: 121	 steps:121	time:121 18/90
Frames: 121	 steps:121	time:121 19/90
Frames: 120	 steps:120	time:120 20/90
Frames: 134	 steps:134	time:134 21/90
Frames: 133	 steps:133	time:133 22/90
Frames: 129	 steps:129	time:129 23/90
Frames: 133	 steps:133	time:133 24/90
Frames: 133	 steps:133	time:133 25/90
Frames: 133	 steps:133	time:133 26/90
Frames: 132	 steps:132	time:13

# Find Warren Cowley files

In [16]:
pattern = os.path.join(simulation_folder, '**', '*_warren_cowley.csv')
path_wc_files = sorted(glob.glob(pattern, recursive=True))
paths_wc_files_clusterd = {} 
for path in path_wc_files:
    dir_name = os.path.dirname(os.path.dirname(path)).split('/')[-1]
    if dir_name not in paths_wc_files_clusterd:
        paths_wc_files_clusterd[dir_name] = []
    paths_wc_files_clusterd[dir_name].append(path)
print(f"Found {len(path_wc_files)} warren cowley files to process.")

Found 90 warren cowley files to process.


In [38]:
# plots for each warren cowley file
for i, path in enumerate(path_wc_files):
    progress_print(i, len(path_wc_files), 'single wc plots')
    
    # Read data
    dataset = pd.read_csv(path, sep=r',\s*', engine='python')
    dataset.columns = dataset.columns.str.strip()
    
    # Create figure
    plt.figure(figsize=(10, 6))
    
    # Seaborn regression plot with scatter and confidence interval
    # Using lowess=True for smooth non-linear curve (locally weighted regression)
    sns.regplot(x='time', y='Li-X', data=dataset, 
                scatter_kws={'alpha':0.5, 's':20},
                line_kws={'color':'red', 'linewidth':2},
                lowess=True)
    
    plt.xlabel('Time')
    plt.ylabel('Li-X')
    plt.title('Warren-Cowley Parameter: Li-X vs Time')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(path.replace('.csv', '_Li-X_warren_cowley.png'), dpi=150)
    plt.close()
    
# plots for clustered warren cowley files
for i, (cluster_name, paths) in enumerate(paths_wc_files_clusterd.items()):
    progress_print(i, len(paths_wc_files_clusterd), f'clustered wc plots')
    plt.figure(figsize=(10, 6))
    
    # Combine all datasets from the cluster into one
    all_datasets = []
    for run_num, path in enumerate(paths):
        dataset = pd.read_csv(path, sep=r',\s*', engine='python')
        dataset.columns = dataset.columns.str.strip()
        dataset['run'] = f'Run {run_num+1}'
        all_datasets.append(dataset)
    
    # Concatenate all data into single dataframe
    combined_dataset = pd.concat(all_datasets, ignore_index=True)
    
    # Plot scatter with color by run
    sns.scatterplot(x='time', y='Li-X', hue='run', data=combined_dataset, 
                    alpha=0.5, s=20)
    
    # Add single regression line (without hue)
    sns.regplot(x='time', y='Li-X', data=combined_dataset, 
               scatter=False,
               line_kws={'color':'red', 'linewidth':2},
               lowess=True)
    
    plt.xlabel('Time')
    plt.ylabel('Li-X')
    plt.title(f'Warren-Cowley Parameter: Li-X vs Time for Cluster {cluster_name}')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(simulation_folder, f'cluster_{cluster_name}_Li-X_warren_cowley.png'), dpi=150)
    plt.close()
